In [1]:
!gpustat

ubuntu                      Tue Apr  4 23:21:32 2023  510.47.03
[0] NVIDIA GeForce RTX 3090 | 30'C,   0 % |   308 / 24576 MB |
[1] NVIDIA GeForce RTX 3090 | 54'C,  14 % | 16032 / 24576 MB | yuzhenqin(15719M)
[2] NVIDIA GeForce RTX 3090 | 29'C,   0 % |   308 / 24576 MB |
[3] NVIDIA GeForce RTX 3090 | 28'C,   0 % |   308 / 24576 MB |
[4] NVIDIA GeForce RTX 3090 | 54'C,   0 % | 14812 / 24576 MB | yuzhenqin(14499M)
[5] NVIDIA GeForce RTX 3090 | 32'C,   0 % |   308 / 24576 MB |
[6] NVIDIA GeForce RTX 3090 | 51'C,   0 % | 13546 / 24576 MB | yuzhenqin(13233M)
[7] NVIDIA GeForce RTX 3090 | 30'C,   0 % |   308 / 24576 MB |


In [ ]:
# 库函数的调用
from models.DG_models import generate_model
from datasets import train_loader,val_loader,test_loader
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as pltac
import os
os.environ["CUDA_VISIBLE_DEVICES"] ='2'
device='cuda:0'
import wfdb
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from scipy import signal
from tqdm import tqdm
import math
import time
import torch
from torch import nn
from torchsummary import summary
import sklearn.metrics
import csv
from utils import *

warm_epoch=2
total_epoch=20
base_lr=2e-3
batch_size=128
max_lr=base_lr*batch_size/256
min_lr=max_lr/100
numleads=1
# domain generalization
DG_method=None
# None #'DG_GR' #'MMD' # 'CORAL'  # 'IRM' # 'CausIRL_MMD' # 'CausIRL_CORAL' # 'cls_awr_ali' # 'DG_GR+IRM' 
# 'DG_GR_ensemble' # 'origin_add_fft_amp' # 'origin_add_fft_phase' # 'origin_add_fft_amp_phase' # 'origin_add_fft_amp_DGGR'
# 'DGGR_smooth' #'origin_add_fft_amp_DGGR_smooth'

add_recons=None #'fft','ksqi','psqi','ssqi','bass',None
ensemble_recons=['fft','ksqi'] # ['fft','ksqi','psqi','ssqi','bass'] 
MMD_iter=4
MMD_gamma=1.
irm_lambda=1e2
irm_penalty_anneal_iters=10000000 # don't penalty #500
DGGR_smooth_eps=0.4
ali_weight=0.5
train_val_test_split=[0.8,0.2,1] # 1 for inter
smote=False
SAM_flag=False
base_optimizer=torch.optim.Adam
loss_fn=nn.CrossEntropyLoss()
# cnn_Ag
# xresnet18,xresnet34,xresnet50,xresnet101,xresnet152
# inception1d
# resnet18,resnet34,resnet50,resnet101,resnet152
# resnet1d_wang
# seresnet_big_18,seresnet_big_34,seresnet_big_50,seresnet_big_101,seresnet_big_152
# densenet1d

base_model='cnn_Ag'
exp_num=10
if DG_method in ['DGGR_smooth','origin_add_fft_amp_DGGR_smooth'] :
    param_flag='smote-'+str(smote)+'-SAM-'+str(SAM_flag)+'-DG_method-'+str(DG_method)+'-eps-'+str(int(DGGR_smooth_eps*100))
else:
    param_flag='smote-'+str(smote)+'-SAM-'+str(SAM_flag)+'-DG_method-'+str(DG_method)

print('ok')

# 10次实验取平均

auc_dict_list=[]
pre_dict_list=[]
rec_dict_list=[]
f1_dict_list=[]
f1_normal_dict_list=[]
f1_AF_dict_list=[]
f1_other_dict_list=[]
acc_dict_list=[]

for exp_idx in range(exp_num):
    
    auc_dict={'T':{},'A':{},'L':{},'N':{},'H':{},'B':{}}
    pre_dict={'T':{},'A':{},'L':{},'N':{},'H':{},'B':{}}
    rec_dict={'T':{},'A':{},'L':{},'N':{},'H':{},'B':{}}
    f1_dict={'T':{},'A':{},'L':{},'N':{},'H':{},'B':{}}
    f1_normal_dict={'T':{},'A':{},'L':{},'N':{},'H':{},'B':{}}
    f1_AF_dict={'T':{},'A':{},'L':{},'N':{},'H':{},'B':{}}
    f1_other_dict={'T':{},'A':{},'L':{},'N':{},'H':{},'B':{}}
    acc_dict={'T':{},'A':{},'L':{},'N':{},'H':{},'B':{}}

    full_list=['T','A','L','N','H','B']

    for jj in full_list:
        ii=list(set(full_list)-set(jj))
        train_data_list=ii    # T,A,L,N,H,B       
        val_data_list=ii
        test_data_list=[jj]
        
        train_datasloader=train_loader(dataname=train_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,smote=smote).loader()
        val_datasloader=val_loader(dataname=val_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2).loader()
        test_datasloader=test_loader(dataname=test_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2).loader()

        recons_model=None
        if DG_method=='DG_GR_ensemble':
            model={}
            optimizer={}
            recons_model={}
            model['origin']=generate_model(base_model=base_model,input_channels=numleads, num_classes=3,DG_method=DG_method,domain_classes=len(train_data_list)).to(device)
            optimizer['origin']=generate_optimizer(base_optimizer=base_optimizer,model_parameters=model['origin'].parameters(),SAM_flag=SAM_flag,weight_decay=1e-4)
            for model_type in ensemble_recons:
                model[model_type]=generate_model(base_model=base_model,input_channels=numleads, num_classes=3,DG_method=DG_method,domain_classes=len(train_data_list)).to(device)
                recons_model_name=find_pretrained_model_name(train_data_list,model_type)
                recons_model[model_type]=torch.load(recons_model_name).to(device)
                recons_model[model_type].eval()
                optimizer[model_type]=generate_optimizer(base_optimizer=base_optimizer,model_parameters=model[model_type].parameters(),SAM_flag=SAM_flag,weight_decay=1e-4)

        elif add_recons:
            recons_model_name=find_pretrained_model_name(train_data_list,add_recons)
            recons_model=torch.load(recons_model_name).to(device)
            recons_model.eval()
            model=generate_model(base_model=base_model,input_channels=numleads*2, num_classes=3,DG_method=DG_method,domain_classes=len(train_data_list)).to(device)
            optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=model.parameters(),SAM_flag=SAM_flag,weight_decay=1e-4)
        elif DG_method in ['origin_add_fft_amp','origin_add_fft_phase','origin_add_fft_amp_DGGR','origin_add_fft_amp_DGGR_smooth']:
            model=generate_model(base_model=base_model,input_channels=numleads*2, num_classes=3,DG_method=DG_method,domain_classes=len(train_data_list)).to(device)
            optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=model.parameters(),SAM_flag=SAM_flag,weight_decay=1e-4)
        elif DG_method=='origin_add_fft_amp_phase':
            model=generate_model(base_model=base_model,input_channels=numleads*3, num_classes=3,DG_method=DG_method,domain_classes=len(train_data_list)).to(device)
            optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=model.parameters(),SAM_flag=SAM_flag,weight_decay=1e-4)
        else:
            model=generate_model(base_model=base_model,input_channels=numleads, num_classes=3,DG_method=DG_method,domain_classes=len(train_data_list)).to(device)
            optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=model.parameters(),SAM_flag=SAM_flag,weight_decay=1e-4)
            
        best_val_acc = 0.0
        best_model_weights = None
        
        for epoch in range(total_epoch):
            print('_____________________________________')
            print(' ')
            time.sleep(0.1)
            train_one_epoch(epoch, warm_epoch, total_epoch, model,max_lr,min_lr, loss_fn, optimizer, \
                            base_optimizer, train_datasloader, device, \
                             DG_method=DG_method,MMD_iter=MMD_iter,MMD_gamma=MMD_gamma, \
                             irm_lambda=irm_lambda,irm_penalty_anneal_iters=irm_penalty_anneal_iters, \
                            ali_weight=ali_weight,SAM_operate=SAM_flag,recons_model=recons_model,add_recons=add_recons,  \
                           DGGR_smooth_eps=DGGR_smooth_eps)
            time.sleep(0.1)
            _,_,_,_,_,_,_,acc_val=val_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, val_datasloader, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons) 
            print(acc_val)
            
            if int(acc_val) > best_val_acc:
                best_val_acc = int(acc_val)
                best_model_weights = model.state_dict()  # Save the current model weights
            
        
        # Load the best model weights before testing
        model.load_state_dict(best_model_weights)

        auc_dict[jj],pre_dict[jj],rec_dict[jj],f1_dict[jj],f1_normal_dict[jj],f1_AF_dict[jj],f1_other_dict[jj],acc_dict[jj]=  \
            test_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, test_datasloader, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons) 
        
    auc_dict_list.append(auc_dict)    
    pre_dict_list.append(pre_dict)  
    rec_dict_list.append(rec_dict)  
    f1_dict_list.append(f1_dict)  
    f1_normal_dict_list.append(f1_normal_dict)  
    f1_AF_dict_list.append(f1_AF_dict)  
    f1_other_dict_list.append(f1_other_dict)  
    acc_dict_list.append(acc_dict)  

auc_dict=avg_dict(auc_dict_list)
pre_dict=avg_dict(pre_dict_list)
rec_dict=avg_dict(rec_dict_list)
f1_dict=avg_dict(f1_dict_list)
f1_normal_dict=avg_dict(f1_normal_dict_list)
f1_AF_dict=avg_dict(f1_AF_dict_list)
f1_other_dict=avg_dict(f1_other_dict_list)
acc_dict=avg_dict(acc_dict_list)

# 写入csv

header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'ALNHB',
        auc_dict['T'],pre_dict['T'],rec_dict['T'],f1_dict['T'],f1_normal_dict['T'],f1_AF_dict['T'],f1_other_dict['T'],acc_dict['T']+'\t',
         ],
        [
        'TLNHB',
        auc_dict['A'],pre_dict['A'],rec_dict['A'],f1_dict['A'],f1_normal_dict['A'],f1_AF_dict['A'],f1_other_dict['A'],acc_dict['A']+'\t',
         ],
        [
        'TANHB',
        auc_dict['L'],pre_dict['L'],rec_dict['L'],f1_dict['L'],f1_normal_dict['L'],f1_AF_dict['L'],f1_other_dict['L'],acc_dict['L']+'\t',
         ],
        [
        'TALHB',
        auc_dict['N'],pre_dict['N'],rec_dict['N'],f1_dict['N'],f1_normal_dict['N'],f1_AF_dict['N'],f1_other_dict['N'],acc_dict['N']+'\t',
         ],
        [
        'TALNB',
        auc_dict['H'],pre_dict['H'],rec_dict['H'],f1_dict['H'],f1_normal_dict['H'],f1_AF_dict['H'],f1_other_dict['H'],acc_dict['H']+'\t',
         ],
        [
        'TALNH',
        auc_dict['B'],pre_dict['B'],rec_dict['B'],f1_dict['B'],f1_normal_dict['B'],f1_AF_dict['B'],f1_other_dict['B'],acc_dict['B']+'\t',
         ],
        ]

with open('log/DGvary_shuffle_multi-one-addrecons-'+str(add_recons)+'-'+str(exp_num)+'timesmean-'+param_flag+'-'+base_model+'-'+str(int(time.time()))+'.csv','w',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)

ok
train data B
train data size : 5501
train label class :  [ 742.  859. 3900. 5501.    0.    0.    0.    0.]
train data N
train data size : 8516
train label class :  [1462. 1769. 5285.    0. 8516.    0.    0.    0.]
train data H
train data size : 20616
train label class :  [11152.   557.  8907.     0.     0. 20616.     0.     0.]
train data A
train data size : 8274
train label class :  [1393.  480. 6401.    0.    0.    0. 8274.    0.]
train data L
train data size : 16824
train label class :  [13511.  1158.  2155.     0.     0.     0.     0. 16824.]
total train data shape =  (59731, 1000, 1)
total train label shape =  (59731, 8)
total train data class =  [28260.  4823. 26648.  5501.  8516. 20616.  8274. 16824.]
val data B
val data size : 1376
val label class :  [176. 239. 961.]
val data N
val data size : 2130
val label class :  [ 364.  456. 1310.]
val data H
val data size : 5154
val label class :  [2753.  118. 2283.]
val data A
val data size : 2069
val label class :  [ 359.   90. 1620.

epoch 0 loss: 0.5038: 100%|████████████████████████████████████| 117/117 [00:02<00:00, 57.07it/s]


780
_____________________________________
 


epoch 1 loss: 0.6387: 100%|████████████████████████████████████| 117/117 [00:02<00:00, 50.46it/s]


713
_____________________________________
 


...

epoch 2 loss: 0.4336: 100%|██████████████████████████████████████| 97/97 [00:01<00:00, 52.62it/s]


815
_____________________________________
 


epoch 3 loss: 0.4464: 100%|██████████████████████████████████████| 97/97 [00:01<00:00, 52.24it/s]


809
_____________________________________
 


epoch 4 loss: 0.4300:  22%|████████                             | 84/387 [00:04<00:16, 17.95it/s]